# SDP Data Flow Graph Generation and Analysis

(Last tested with IPython 4.0.0 using Python 2.7 and GraphViz 2.36 - If you have trouble, check version compatibility)

In [ ]:
""" These first few lines below import the IPython definitions and methods that we will use. 
Please Refer to ipython_api.py for the implementation """
from ipywidgets import interact, interact_manual, fixed, SelectMultiple
from IPython.display import display, SVG

from pipeline import Pipeline, flowsToDot
from parameter_definitions import *
from implementation import Implementation as imp, PipelineConfig
%matplotlib inline

# Make a configuration
cfg = PipelineConfig(telescope=Telescopes.SKA1_Mid,
                     band=Bands.Mid1,
                     mode=ImagingModes.Continuum,
                     max_baseline='default',
                     Nf_max='default',
                     bldta=True,
                     on_the_fly=False)

# Calculate optimised parameters
tp = imp.calc_tel_params(cfg)
(tsnap_opt, nfacet_opt) = imp.find_optimal_Tsnap_Nfacet(tp)
tp = imp.calc_tel_params(cfg, adjusts={ 'Nfacet': nfacet_opt, 'Tsnap': tsnap_opt })

# Generate DOT and render
Trun = 4 * 3600
print "Assuming a pipeline run time of %02d:%02d:" % (Trun / 60, Trun % 60)
dot = flowsToDot(Pipeline(tp).create_continuum(), tp.Tobs,
                 graph_attr={'size':'10,100'},
                 node_attr={'shape':'box', 'style':'filled','color':'lightgrey'})
display(SVG(dot._repr_svg_()))

In [ ]:
# Export, if desired
dot.format='pdf' # png, svg, ...
dot.render('out')